# Projet maison n° 4

In [13]:
# imports
import requests
from bs4 import BeautifulSoup 

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [2]:
# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    
    r = requests.get(url)
    bs = BeautifulSoup(r.content)
    divs_tag = bs.findAll('div', attrs = {'class': 'pricing-table'})
    
    # On récupère les prix 
    dollars = []
    for elem in divs_tag:
        string = elem.find('div', attrs = {'class' :'pricing-table-header'}).span.text
        cost = string[string.find('$'): string.find('p')-1]
        dollars.append(cost)
    
    # Construction du dictionnaire
    prices = \
        { 
            dt.find('h2').text : 

            { 
                'price' : dollars[k],
                'storage' : dt.findAll('li')[3].text.split(' ')[0],
                'databases' : int(dt.findAll('li')[4].text.split(' ')[0])
            }

            for k, dt in enumerate(divs_tag)
        }

    
    return prices

In [5]:
get_prices_from_url(URL_PAGE2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}

In [25]:
import re as reg
# partie B-1
def extract_beer_list_infos(url):
    
    # Parse en JSON 
    response = requests.get(url)
    data = response.json()
    
    list_brasseurs = []
    pays_origines = []
    url_bieres = [item['contentReference'] for item in data['items']]

    # Web Scrapling pour récupéré la valeur de la clé "Brasseur" et "Origine" qui ne sont pas disponible dans les données JSON
    for i in range(len(url_bieres)):
        response = requests.get("https://www.beerwulf.com/"+url_bieres[i])
        bs = BeautifulSoup(response.content)
        txt_brasseur = bs.findAll('dd', {'class' : 'small-6 medium-9 columns'})[-1].text
        origine_pays = bs.find('dd', {'class' : 'small-6 medium-9 columns js-beer-country'}).attrs['data-country']
        words_brasseur = reg.findall('[A-Za-z]+', txt_brasseur)
        brasseur = ' '.join(words_brasseur)
        list_brasseurs.append(brasseur)
        pays_origines.append(origine_pays)


        # On récupère le pays d'origine 
    features_item = data['items']
    f = data['filters']
    
    beer_infos = [ 
        {'Nom' : f['title'],
        'Style' : f['style'],
        'Contenu' : int(f['details'].split(' ')[-2]),
        "Degré d’alcool" : float(f['details'].split('|')[1][:-1].strip()[:-1].replace(',','.')),
        'Origine' : pays_origines[k],
        'Brasseur' : list_brasseurs[k]}
        for k,(f,p,l) in enumerate(zip(features_item, pays_origines, list_brasseurs))]
    
   
    return beer_infos

In [26]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [27]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 1.379s

OK
